<a href="https://colab.research.google.com/github/SatoshiEndoDEV/SatoshiEndoDEV/blob/main/nishika_fakenews2022_bert_v2_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

udemyコース

early stopping追加

stratified k fold追加+randomstate+sigmoid

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
data_train = pd.read_csv('/content/drive/MyDrive/nishika/fakenews2022/data/train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/nishika/fakenews2022/data/test.csv')

In [4]:
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 4.0 MB 37.6 MB/s 
     |████████████████████████████████| 6.6 MB 87.5 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 880 kB 76.0 MB/s 
     |████████████████████████████████| 596 kB 83.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4d40e0009a1a11276a8e76ab9ff7575bd543011dc73792944f3b3b10f64e4914
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 30.5 MB/s 
     |████████████████████████████████| 212 kB 81.7 MB/s 
     |████████████████████████████████| 325 kB 29.3 MB/s 
     |████████████████████████████████| 1.1 MB 74.6 MB/s 
     |████████████████████████████

In [5]:
!pip install unidic_lite

     |████████████████████████████████| 47.4 MB 59.4 MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=eede0dfc86fe44b5470257be232ec7b7b6160fe66626291a859e99044dfc22f7
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


In [6]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [7]:
data_train.drop('id', axis=True, inplace=True)
df_test = data_test.drop('id', axis=True)

In [8]:
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch
from tqdm.notebook import tqdm

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)


from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

In [9]:
training_args = TrainingArguments(
    output_dir = "./results",
    #f"training_with_callbacks",
    evaluation_strategy ='steps',
    eval_steps = 50, # Evaluation and Save happens every 50 steps
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    num_train_epochs = 50,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 50,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ありません
    logging_dir = "./logs",
    load_best_model_at_end=True,
)

In [10]:
max_length = 512

In [11]:
pred_list_all = []
oof_data_train = np.zeros((len(data_train),2))

for fold, (train_index, val_index) in enumerate(skf.split(data_train, data_train['isFake'])):
    print(f'---------- {fold+1} ----------')

    # train/val 分割
    df_train = data_train.loc[train_index,:].copy()
    df_val = data_train.loc[val_index,:].copy()

    # いったんcsvにおとす
    df_train.to_csv('sp_train.csv', index=False, header=False)
    df_val.to_csv('sp_val.csv',index=False, header=False)

    # model, tokenizer
    sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-v2", num_labels=2)
    sc_model.cuda()
    tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")

    # dataload
    train_data = load_dataset("csv", data_files="sp_train.csv", column_names=["label","text"], split="train")
    train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
    train_data.set_format("torch", columns=["input_ids", "label"])

    val_data = load_dataset("csv", data_files="sp_val.csv", column_names=["label","text"], split="train")
    val_data = val_data.map(tokenize, batched=True, batch_size=len(val_data))
    val_data.set_format("torch", columns=["input_ids", "label"])

    # trainer
    trainer = Trainer(
        model = sc_model,
        args = training_args,
        compute_metrics = compute_metrics,
        train_dataset = train_data,
        eval_dataset = val_data,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=5)],
        )
    
    # train 
    trainer.train()

    # evaluation
    trainer.evaluate()

    # testの予測
    pred_list = []
    for _, text in enumerate(tqdm(data_test['text'])):
        word = tokenizer.tokenize(text)
        word_ids = tokenizer.convert_tokens_to_ids(word)
        word_tensor = torch.tensor([word_ids[:max_length]]) 
        
        x = word_tensor.cuda()  # GPU対応
        y = sc_model(x)  # 予測
        pred = y[0]#.argmax(-1)  # 最大値のインデックス
        pred = pred.to('cpu').detach().numpy().copy()
        pred_list.extend(pred)
    
    # 1fold分の予測値を追加
    pred_list_all.append(pred_list)

    # oofの予測
    oof_list = []
    for _, text in enumerate(tqdm(df_val['text'])):
        word = tokenizer.tokenize(text)
        word_ids = tokenizer.convert_tokens_to_ids(word)
        word_tensor = torch.tensor([word_ids[:max_length]]) 
        
        x = word_tensor.cuda()  # GPU対応
        y = sc_model(x)  # 予測
        pred = y[0]#.argmax(-1)  # 最大値のインデックス
        pred = pred.to('cpu').detach().numpy().copy()
        oof_list.extend(pred)
    
    # 1fold分のval予測値を追加
    oof_data_train[val_index] = oof_list

---------- 1 ----------


Downloading:   0%|          | 0.00/517 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/427M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

...

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3025
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18950


Step,Training Loss,Validation Loss,Accuracy
50,No log,0.513970,0.777778
100,No log,0.613975,0.744709
150,No log,0.169554,0.936508
200,No log,0.167658,0.937831
250,No log,0.126082,0.957672
300,No log,0.223561,0.939153
350,No log,0.208627,0.949735
400,No log,0.112001,0.965608
450,No log,0.118832,0.969577
500,0.272500,0.094198,0.972222


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 756
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 756
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 756
  Batch size = 32
The following

  0%|          | 0/3781 [00:00<?, ?it/s]

  0%|          | 0/756 [00:00<?, ?it/s]

# 評価

In [12]:
# シグモイド関数の定義
def sigmoid(a):
    return 1 / (1 + np.exp(-a))

In [13]:
oof = sigmoid(oof_data_train)
oof = np.argmax(oof, axis=-1) 

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(data_train['isFake'].values, oof)

0.976725733932822

In [15]:
pred_list_all = np.array(pred_list_all)
y_pred_submit = sigmoid(pred_list_all)
y_pred_submit = np.sum(y_pred_submit, axis=0) / len(y_pred_submit)
y_pred_submit

array([[0.9929439 , 0.00814354],
       [0.3929078 , 0.59787446],
       [0.01695639, 0.98238575],
       ...,
       [0.30718428, 0.6871127 ],
       [0.01847059, 0.98242444],
       [0.01421475, 0.9850518 ]], dtype=float32)

In [16]:
submit = np.argmax(y_pred_submit, axis=-1) 

In [17]:
submit

array([0, 1, 1, ..., 1, 1, 1])

psuedolabeling用データフレーム

In [44]:
df_y = pd.DataFrame(y_pred_submit, columns=['0', '1'])
df_l = pd.DataFrame(submit, columns=['isFake'])

In [46]:
df_result = pd.concat([df_y, df_l], axis=1)

In [48]:
df_result.to_csv('/content/drive/MyDrive/nishika/fakenews2022/bert_v2_sfkfold-3_prob.csv', index=False)

# サブミット

In [49]:
data_test['isFake'] = submit

In [50]:
sub = data_test.loc[:,['id', 'isFake']].copy()

In [51]:
sub.to_csv('/content/drive/MyDrive/nishika/fakenews2022/bert_v2_sfkfold-3.csv', index=False)